# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントを開発することを目的としています。このエージェントは、質問を行い、回答を受け取り、推測をします。主に、次のような問題に取り組んでいます：

1. **モデルのセットアップと接続**: Hugging Faceのモデル（Meta-Llama-3-8B-Instruct-hf-AWQ）をダウンロードし、vLLMを用いてAPIサーバーを起動します。
2. **質問者エージェントの実装**: 質問を行うためのロジック、回答を処理するためのロジック、推測を生成するためのロジックがそれぞれ実装されています。

使用されている主な手法とライブラリは以下の通りです：
- **Hugging Face Hub**: モデルのスナップショットをダウンロードするために使用。
- **vLLM**: 高速化を図るためのモデルサーバー。
- **Rigging**: エージェントが質問を生成し、回答を解析するためのライブラリ。
- **Pydantic**: データモデルを定義するのに使用し、各種質問、回答、推測を構造化しています。

Notebookは、依存関係のインストール、モデルのダウンロード、vLLMサーバーの起動、質問・回答・推測のロジックを含むメインエージェントの実装を行い、最終的にKaggleコンペティションへの提出を行うコードから構成されています。また、ユーザーのシークレット情報を扱うためのコードも含まれており、KaggleのAPIを介して提出を行うことも可能です。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の簡単な解説です。特にマイナーなものや実務経験がないと馴染みのないものを中心に説明します。

### 専門用語の解説

1. **Hugging Face トークン (HF_TOKEN)**:
   Hugging Faceは、多くの深層学習モデル（特に自然言語処理のモデル）を提供するプラットフォームです。HF_TOKENはそのサービスにアクセスするための認証用トークンで、ユーザーが自分のアカウントにアクセスしてモデルをダウンロードしたり、APIを利用したりする際に使用されます。

2. **rigging**:
   これは、AIモデルのサービングや生成に関連するライブラリです。riggingは、さまざまなLLM（大規模言語モデル）を管理し、効率的に利用するための機能を提供します。特にAPI経由でモデルを呼び出す際に便利です。

3. **vllm**:
   vllmは、AIモデルを高速でサーブ（提供）するためのフレームワークです。このライブラリは、特に大規模な言語モデルの性能を良く引き出すために設計されています。vllmを使用すると、効率的にリクエストを処理し、応答を生成することができます。

4. **シンボリックリンク (symlink)**:
   シンボリックリンクは、ファイルシステム内で他のファイルやディレクトリへの参照を示す特別なタイプのファイルです。シンボリックリンクを使用すると、実際のファイルの場所を移動しても、リンクを通じてそのファイルにアクセスできるため、便利です。

5. **サーバーリッスンポート**:
   サーバーが外部からのリクエストを受け付けるために使用するポートです。このポート番号は、サーバーが特定のリクエスト（例えばHTTPトラフィック）を待ち受ける際に必要です。このノートブックでは、ポート9999がvllmのために指定されています。

6. **Pydantic**:
   Pydanticは、データの検証と設定を行うためのPythonライブラリです。特に、型ヒントを用いてデータモデルを定義し、そのデータが正しいかどうかを検証する際に使用されます。Pythonのデータクラスに似ていますが、追加のバリデーション機能を提供します。

7. **XML形式の例 (xml_example)**:
   XML（拡張マークアップ言語）は、データを構造的に表現するためのフォーマットです。`xml_example`メソッドは、特定のモデルがどのようにXML形式で表現されるべきかを示すサンプルを提供します。

8. **サーバーサイドでの処理 (APIサーバー実行)**:
   APIサーバーは、クライアントからのリクエストに対して応答を返すためのソフトウェアです。ノートブック内のvllmは、このAPIサーバーとして機能し、他のプログラムやサービスからの要求に対してAIモデルを通じて応答を生成します。

これらの説明は、機械学習や深層学習の初心者が遭遇しそうな専門用語の理解を助けるためのものです。他の一般的な用語やコンセプトについては、知識をお持ちのことと考えますので省略しました。

---


In [ ]:
# シークレット（オプション）

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

# HF_TOKENを文字列またはNoneで初期化します
HF_TOKEN: str | None  = None
# KAGGLE_KEYを文字列またはNoneで初期化します
KAGGLE_KEY: str | None = None
# KAGGLE_USERNAMEを文字列またはNoneで初期化します
KAGGLE_USERNAME: str | None = None
    
try:
    # シークレットからHF_TOKENを取得します
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    # シークレットからKAGGLE_KEYを取得します
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    # シークレットからKAGGLE_USERNAMEを取得します
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    # エラーが発生した場合は何もしません（パスします）
    pass

In [ ]:
# 依存関係（速度向上のためのuv）

# uvをインストールします
!pip install uv

# riggingとkaggleをアップグレードインストールします
!uv pip install -U \
    --python $(which python) \  # 実行中のPythonのパスを指定します
    --target /kaggle/tmp/lib \  # インストール先ディレクトリを指定します
    rigging==1.3.0 \  # riggingパッケージのバージョン1.3.0を指定します
    kaggle

# vllmをアップグレードインストールします
!uv pip install -U \
    --python $(which python) \  # 実行中のPythonのパスを指定します
    --target /kaggle/tmp/srvlib \  # 別のインストール先ディレクトリを指定します
    vllm  # vllmパッケージをインストールします

In [ ]:
# モデルのダウンロード

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

# モデルを保存するパスを指定します
g_model_path = Path("/kaggle/tmp/model")
# 既にモデルのパスが存在する場合は、そのディレクトリを削除します
if g_model_path.exists():
    shutil.rmtree(g_model_path)
# 新しいディレクトリを作成します
g_model_path.mkdir(parents=True)

# モデルのスナップショットをダウンロードします
snapshot_download(
    repo_id="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",  # ダウンロードするモデルのIDを指定します
    ignore_patterns="original*",  # 無視するファイルパターンを指定します
    local_dir=g_model_path,  # ダウンロード先のローカルディレクトリを指定します
    local_dir_use_symlinks=False,  # シンボリックリンクは使用しません
    token=globals().get("HF_TOKEN", None)  # Hugging Faceのトークンを取得します
)

In [ ]:
%%writefile util.py

# vLLMサーバーを開始するためのヘルパー関数

import subprocess
import os
import socket
import time

# ポートが使用中かどうかを確認する関数
def check_port(port: int) -> bool:
    try:
        # ソケットを作成してポートをチェックします
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)  # タイムアウトを1秒に設定
            result = sock.connect_ex(('localhost', port))  # 指定したポートに接続を試みる
            if result == 0:  # 接続成功（ポートがオープン）
                return True
    except socket.error:
        pass  # エラーが発生した場合は無視
    
    return False  # ポートがオープンでない場合

# プロセスを実行し、指定したポートが開くのを待つ関数
def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60
) -> subprocess.Popen:
    
    if check_port(port):
        # 既にポートが開いている場合はエラーを返します
        raise ValueError(f"ポート {port} はすでに開いています")
        
    # コマンドを実行します
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},  # 環境変数を結合します
        stdout=subprocess.DEVNULL,  # 標準出力を無視します
        stderr=subprocess.DEVNULL   # 標準エラーを無視します
    )
    
    start_time = time.time()
    # タイムアウトまでポートが開くのを待ちます
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen  # ポートが開いた場合、プロセスを返します
        time.sleep(1)  # 1秒待ちます
    
    popen.terminate()  # タイムアウトした場合、プロセスを終了します
    raise Exception(f"プロセスは {timeout} 秒以内にポート {port} を開きませんでした。")

In [ ]:
# vLLMの起動を検証する（オプション - ビルドを速くするためにコメントアウト）

# import importlib
# from pathlib import Path
# import util

# util = importlib.reload(util)  # utilモジュールを再読み込みします

# g_srvlib_path = Path("/kaggle/tmp/srvlib")  # srvlibのパスを指定します
# assert g_srvlib_path.exists()  # パスが存在することを確認します

# g_model_path = Path("/kaggle/tmp/model")  # モデルのパスを指定します
# assert g_model_path.exists()  # パスが存在することを確認します

# g_vllm_port = 9999  # vLLMがリッスンするポートを指定します
# g_vllm_model_name = "custom"  # サーブするモデルの名前を指定します

# # vLLMサーバーを開始します

# vllm = util.run_and_wait_for_port([
#     "python", "-m",  # Pythonモジュールとして実行します
#     "vllm.entrypoints.openai.api_server",  # vLLMのAPIサーバーを指定します
#     "--enforce-eager",  # 即時実行を強制します
#     "--model", str(g_model_path),  # モデルのパスを指定します
#     "--port", str(g_vllm_port),  # 使用するポートを指定します
#     "--served-model-name", g_vllm_model_name  # サーブされるモデルの名前を指定します
# ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})  # ポートが開くのを待ちながら実行します

# print("vLLMが起動しました")  # サーバーが正常に起動したことを表示します

In [ ]:
# Riggingに接続します（オプション - ビルドを速くするためにコメントアウト）

# import sys
# import logging

# sys.path.insert(0, "/kaggle/tmp/lib")  # ライブラリパスを追加します

# logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ロギングのレベルをWARNINGに設定します

# import rigging as rg  # riggingモジュールをインポートします

# generator = rg.get_generator(
#     f"openai/{g_vllm_model_name},"  # 使用するモデル名を指定します
#     f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIベースURLを指定します
#     "api_key=sk-1234,"  # APIキーを指定します
#     "stop=<|eot_id|>"  # Llamaモデルでは特別な停止トークンが必要です
# )
# chat = generator.chat("Say Hello!").run()  # "Say Hello!"というメッセージでチャットを開始します

# print(chat.last)  # チャットの最後の応答を表示します

In [ ]:
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

# パスの修正

g_working_path = Path('/kaggle/working')  # 作業ディレクトリのパス
g_input_path = Path('/kaggle/input')  # 入力データのパス
g_temp_path = Path("/kaggle/tmp")  # 一時ファイルのパス
g_agent_path = Path("/kaggle_simulations/agent/")  # エージェントのパス

g_model_path = g_temp_path / "model"  # モデルのパス
g_srvlib_path = g_temp_path / "srvlib"  # サーバーライブラリのパス
g_lib_path = g_temp_path / "lib"  # ライブラリのパス

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"  # エージェントパスが存在する場合、ライブラリパスを更新
    g_model_path = g_agent_path / "model"  # モデルパスを更新
    g_srvlib_path = g_agent_path / "srvlib"  # サーバーライブラリのパスを更新

sys.path.insert(0, str(g_lib_path))  # ライブラリパスをシステムパスに追加

# ロギングの設定

logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログレベルをWARNINGに設定

# インポート

import util  # 利用するモジュール
import rigging as rg  # riggingモジュールをインポート
from pydantic import BaseModel, field_validator, StringConstraints  # Pydanticライブラリから必要なクラスをインポート

# 定数

g_vllm_port = 9999  # vLLMがリッスンするポート
g_vllm_model_name = "custom"  # 使用するモデル名

g_generator_id = (
    f"openai/{g_vllm_model_name},"  # モデル名を指定
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIベースURLを指定
    "api_key=sk-1234,"  # APIキーを指定
    "stop=<|eot_id|>"  # Llamaモデルでは特別な停止トークンが必要
)

# タイプ

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]  # スペースをトリムした文字列

class Observation(BaseModel):
    step: int  # 手順の番号
    role: t.Literal["guesser", "answerer"]  # プレイヤーの役割
    turnType: t.Literal["ask", "answer", "guess"]  # プレイヤーのターンのタイプ
    keyword: str  # キーワード
    category: str  # カテゴリー
    questions: list[str]  # 質問のリスト
    answers: list[str]  # 回答のリスト
    guesses: list[str]  # 推測のリスト
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])  # 質問、回答、推測がすべて空かを確認
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")  # 過去の質問、回答、推測をまとめて返す

    def get_history_as_xml(self, *, include_guesses: bool = False) -> str:
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if include_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "まだなし."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]  # 回答は"yes"または"no"

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):  # 有効な回答の検証
                return valid
        raise ValueError("無効な回答です。'yes'または'no'でなければなりません")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no**{Answer.xml_end_tag()}"  # XML形式の例


class Question(rg.Model):
    content: str_strip  # 質問内容

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # XML形式の例


class Guess(rg.Model):
    content: str_strip  # 推測内容

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="**thing/place/person**").to_pretty_xml()  # XML形式の例


# 関数

def ask(base: rg.PendingChat, observation: Observation) -> str:
    if observation.step == 0:
        # 最初の質問をオーバーライドして不可解なバグを修正
        return "私たちは20の質問をプレイしていますか？"
    
    chat = (
        base.fork(
            f"""\
            あなたは現在、次の質問を尋ねています。

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            上記の履歴に基づいて、最も役立つはい/いいえの質問を次の形式で尋ねてください:
            {Question.xml_example()}

            - あなたの回答は、最も多くの情報を収集するための焦点を絞った質問であるべきです
            - 最初は一般的な質問から始めてください
            - 残された検索空間をバイセクトしようと常に試みてください
            - 過去の質問と回答に注意してください

            始める前に、利用可能な場合はゲーム履歴の分析を文書化し、
            その後、質問を書いてください。
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Question).content


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("キーワードが回答者に提供されていません", file=sys.stderr)
        return "yes"  # 不可解なバグを修正するためにオーバーライド
            
    last_question = observation.questions[-1]
    chat = (
        base.fork(
            f"""\
            このゲームの秘伝の言葉は"{observation.keyword}"です [{observation.category}]

            あなたは上記の言葉に関する質問に答えています。

            次の質問は"{last_question}"です。

            上記のはい/いいえの質問に答え、次の形式で提示してください:
            {Answer.xml_example()}

            - あなたの回答は、上記のキーワードに正確であるべきです
            - 常に"yes"または"no"で答えてください

            答えは何ですか？
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Answer).content


def guess(base: rg.PendingChat, observation: Observation) -> str:
    chat = (
        base.fork(
            f"""\
            あなたは現在、キーワードの情報に基づいた推測を行っています。

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            上記の履歴に基づいて、キーワードに対する次の最良の推測を1つ作成し、次の形式で示してください:
            {Guess.xml_example()}

            - 上記の履歴に基づいて再推測は避けてください
            - 推測は特定の人物、場所、または物であるべきです

            始める前に、利用可能な場合はゲーム履歴の分析を文書化し、
            その後、推測を書いてください。
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)
        .run()
    )
        
    return chat.last.parse(Guess).content

# vLLMとGeneratorの初期化

vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",  # APIサーバーを指定
    "--enforce-eager",  # 即時実行を強制
    "--model", str(g_model_path),  # モデルのパスを指定
    "--port", str(g_vllm_port),  # 使用するポートを指定
    "--served-model-name", g_vllm_model_name  # サーブするモデルの名前を指定
], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

print("vLLMが起動しました")  # サーバーが正常に起動したことを表示

generator = rg.get_generator(g_generator_id)  # Generatorの初期化

base =  generator.chat("""\
あなたは20の質問ゲームの才能あるプレイヤーです。あなたは正確で、焦点を絞り、構造化されたアプローチを持っています。あなたは有用な質問を作成し、推測を行い、またはキーワードに関する質問に答えます。

""")

# エントリーポイント

def agent_fn(obs: t.Any, _: t.Any) -> str:
    observation = Observation(**obs.__dict__)  # 観察オブジェクトを作成
    
    try:
        match observation.turnType:  # プレイヤーのターンのタイプに基づいて処理
            case "ask":
                return ask(base, observation)  # 質問を行う
            case "answer":
                return answer(base, observation)  # 回答を行う
            case "guess":
                return guess(base, observation)  # 推測を行う
            case _:
                raise ValueError("未知のターンタイプです")
    except Exception as e:
        print(str(e), file=sys.stderr)  # エラーを表示
        raise  # エラーを再スロー

In [ ]:
!apt install pigz pv  # pigzとpvをインストールします

In [ ]:
!tar --use-compress-program='pigz --fast' \  # pigzを使って圧縮します（高速モード）
    -cf submission.tar.gz \  # tar.gz形式でアーカイブを作成します
    --dereference \  # シンボリックリンクを解決して参照を保持します
    -C /kaggle/tmp model lib srvlib \  # /kaggle/tmpディレクトリからモデル、ライブラリ、サーバーライブラリを追加します
    -C /kaggle/working main.py util.py  # /kaggle/workingディレクトリからmain.pyとutil.pyを追加します

In [ ]:
!KAGGLE_USERNAME={KAGGLE_USERNAME} \  # Kaggleユーザー名を設定します
 KAGGLE_KEY={KAGGLE_KEY} \  # Kaggle APIキーを設定します
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"  # コンペティションに提出します（"Updates"というメッセージを追加）

---

# コメント 

> ## OminousDude
> 
> こんにちは、あなたのコードをテストしていたのですが、実行したら「AttributeError: 'coroutine' object has no attribute 'last'」という例外が発生して失敗しました。このエラーに遭遇したことはありますか？
> 
> 
> 
> > ## Rob Mulla
> > 
> > [@max1mum](https://www.kaggle.com/max1mum) - これは、以前のバージョンとの互換性がない変更を含むriggingの新しいリリースが原因です。
> > 
> > パッケージの固定バージョンを使用してみてください。インストールセルを次のように変更すれば、動作するはずです。
> > 
> > ```
> > # 依存関係（速度向上のためのuv）
> > !pip install uv==0.1.45
> > 
> > !uv pip install -U \
> >     --python $(which python) \
> >     --target /kaggle/tmp/lib \
> >     rigging==1.1.1 \
> >     kaggle
> > 
> > !uv pip install -U \
> >     --python $(which python) \
> >     --target /kaggle/tmp/srvlib \
> >     vllm==0.4.2
> > 
> > ```
> > 
> > 
> > 
> > > ## OminousDude
> > > 
> > > ありがとうございます!!!
> > > 
> > > 
> > > 


---